In [1]:
!pip install mne
import pickle

In [2]:
with open("seizure_times.pkl","rb") as f:
  seizures=pickle.load(f)

In [3]:
import math as mth
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt

#Parameters
  #n: positive integer representing number of samples
  #t: postive float representing length of time interval [0,t] samples are scaled to (I default to t=1)
  #h: float greater than 0 and less than 1 representing the Hurst Exponent, which uniquely specifies an dBM distribution. h=1/2 yields classic Brownian motion
#Returns:
  #fBM: an (n,1) dimensional nparray of fractional brownian motion samples generated with Hurst exponent h using Cholesky's Method,
  #timeArray: an (n,1) dimensional array of times each sample is taken at
  #h: the input hurst exponent parameter

def FBM(n, t, h):
  CovMat = np.ones((n, n)) #covariance matrix of the Gaussian series (X1 = fB_h(1), X2= fB_h(2)-fB_h(1),..., XN = fB_h(N)-fB_h(N-1)), called fractional Gaussian noise. These are easier to simulate as increments of fBM are stationary

  [Rows, Columns] = np.indices((n,n))
  AbsDiff = np.abs(Rows-Columns)

  ind = np.diag_indices_from(AbsDiff)
  AbsDiff[ind] = 1 #diagonal values don't matter so replace with one to avoid any possible square root of negative one errors

  CovMat = 1/2*(np.power(AbsDiff+1, 2*h) + np.power(AbsDiff-1, 2*h) - 2*np.power(AbsDiff, 2*h)) #covariance matrix of fractional Gaussian noise can be derived from the covariance matrix of fBM(fB_h(i)-fB_h(i-1), fB_h(j)-fB_h(j-1)) = i^2h + j^2h - |i-j|^2h

  ind = np.diag_indices_from(CovMat)
  CovMat[ind] = 1 #variance of each variable is one, so diagonal of covariance matrix must be one

  L = np.linalg.cholesky(CovMat)#covariance matricies are positive definite by construction so CovMat can be decomposed CovMat = LL' where L is lower triangular
  v = np.random.normal(0,1,(n,1)) #vector of independent standard normal variables
  fGN = np.dot(L,v)

  fBM = np.zeros((n, 1))
  timeArray = np.ones((n, 1))
  FBMMat = np.zeros((n,2))

  fBM[0,0] = fGN[0,0]
  for i in range (1,n):
    timeArray[i-1,0] = i*(t/n) # time each sample is taken at in the time interval [0,t]
    fBM[i,0] = fBM[i-1,0] + fGN[i,0]  #calculate Brownian motion as the cumulateive sum of the fGN increments up to that point


  fBM = np.power(t/n,h)*fBM #use self similarity property to scale samples down to a specific time interval

  FBMMat[:,0] = timeArray[:,0]
  FBMMat[:,1] = fBM[:,0]


  return [fBM, timeArray, FBMMat, h]
#Parameters
  #n: positive integer representing number of samples
  #t: postive float representing length of time interval [0,t] samples are scaled to,
  #h: float greater than 0 and less than 1 representing the Hurst Exponent, which uniquely specifies an dBM distribution. h=1/2 yields classic Brownian motion
#Returns:
  #fBM: an (n,1) dimensional nparray of fractional brownian motion samples generated with Hurst exponent h using Cholesky's Method,
  #timeArray: an (n,1) dimensional array of times each sample is taken at
  #h: the input hurst exponent parameter

def FBMFixedSeed(n, t, h, seed):
  CovMat = np.ones((n, n)) #covariance matrix of the Gaussian series (X1 = fB_h(1), X2= fB_h(2)-fB_h(1),..., XN = fB_h(N)-fB_h(N-1)), called fractional Gaussian noise. These are easier to simulate as increments of fBM are stationary

  [Rows, Columns] = np.indices((n,n))
  AbsDiff = np.abs(Rows-Columns)

  ind = np.diag_indices_from(AbsDiff)
  AbsDiff[ind] = 1 #diagonal values don't matter so replace with one to avoid any possible square root of negative one errors

  CovMat = 1/2*(np.power(AbsDiff+1, 2*h) + np.power(AbsDiff-1, 2*h) - 2*np.power(AbsDiff, 2*h)) #covariance matrix of fractional Gaussian noise can be derived from the covariance matrix of E(fB_h(i), fB_h(j)) = i^2h + j^2h - |i-j|^2h

  ind = np.diag_indices_from(CovMat)
  CovMat[ind] = 1 #variance of each variable is one, so diagonal of covariance matrix must be one

  L = np.linalg.cholesky(CovMat)#covariance matricies are positive definite by construction so CovMat can be decomposed CovMat = LL' where L is lower triangular
  fGN = np.dot(L,seed)

  fBM = np.zeros((n, 1))
  timeArray = np.ones((n, 1))
  FBMMat = np.zeros((n,2))

  fBM[0,0] = fGN[0,0]
  for i in range (1,n):
    timeArray[i-1,0] = i*(t/n) # time each sample is taken at in the time interval [0,t]
    fBM[i,0] = fBM[i-1,0] + fGN[i,0]  #calculate Brownian motion as the cumulateive sum of the fGN increments up to that point


  fBM = np.power(t/n,h)*fBM #use self similarity property to scale samples down to a specific time interval

  FBMMat[:,0] = timeArray[:,0]
  FBMMat[:,1] = fBM[:,0]


  return [fBM, timeArray, FBMMat, h, seed]
#Parameters
  #P: nxd ndarray of n d-dimensional datapoints
  #s: real nonegative float s
#Returns:
  #discSEnergy: Float containing the discrete-s energy of P
  #s parameter

def discrete_energy(data, s):
    d_e = 0
    for t1 in range(data.shape[1]):
        for t2 in range(t1 + 1, data.shape[1]):
            x = np.sqrt(np.sum(np.square(data[:, t1] - data[:, t2])))
            d_e += x ** (-1 * s) if x != 0 else 0
    return d_e

def DiscreteSEnergy(P, s):
    n = np.shape(P)[0]

    [Rows, Columns] = np.indices((n,n))

    VecDiff = np.abs(P[Rows,:] - P[Columns,:])

    MagDiff = np.power(np.sum(np.power(VecDiff,2), axis=2),1/2) #here we use L2 norm (Euclidean distance in Rd) to find vector magnitude

    Diff= np.triu(MagDiff) #remove duplicate differences by considering only those differences above the main diagonal
    MaskedDiff = ma.masked_where(Diff==0, Diff, copy=True) #mask all zero differences so we are not dividing by zero

    SumMat = np.ma.power(MaskedDiff, -s)
    sum = np.sum(np.sum(SumMat))

    discSEnergy = np.power(float(n), -2)*sum

    return[discSEnergy, s]


  #Temp = np.abs(P[Rows] - P[Columns])
  #Diff = Temp[:,:,0]
  #Diff= np.triu(Diff) #remove duplicate differences by considering only those differences above the main diagonal
  #MaskedDiff = ma.masked_where(Diff==0, Diff, copy=True) #mask all zero differences so we are not dividing by zero


  #SumMat = np.ma.power(MaskedDiff, -s)
  #sum = np.sum(np.sum(SumMat))

  #discSEnergy = np.power(float(n), -2)*sum

In [ ]:
import mne
import matplotlib.pyplot as plt
import numpy as np

#step 1 - isolate the seizure
#seizing for 1 second
mne.sys_info()
file = "chb01_01.edf"
# file2 = "chb02_19.edf"
# file3 = "chb03_01.edf"

data = mne.io.read_raw_edf(file)
# data2 = mne.io.read_raw_edf(file2)
# data3 = mne.io.read_raw_edf(file3)

raw = data.get_data() #numpy array
# raw2 = data2.get_data()
# raw3 = data3.get_data()

rawsiez = raw[:,1467*256:1494*256] #isolating specific times the patient is seizing, 256 bits of time in a seconds
rawsiez2 = raw2[:,3369*256:3378*256]
rawsiez3 = raw3[:,362*256:414*256]
# you can get the metadata included in the file and a list of all channels:
info = data.info
channels=data.ch_names

info = data2.info
channels=data2.ch_names

info = data3.info
channels=data3.ch_names
#plt.plot(raw[0][:1000])

#discrete energy calculations (step 2)
low_bounds = DiscreteSEnergy(raw, 23)
high_bounds = DiscreteSEnergy(raw, 24)
print(low_bounds[0])
print(high_bounds[0])
test_data = rawsiez / np.max(np.abs(rawsiez)) #normalize
test_data = raw / np.max(np.abs(raw)) #normalize

test_data = rawsiez2 / np.max(np.abs(rawsiez2)) #normalize
test_data = raw2 / np.max(np.abs(raw2)) #normalize
#print(DiscreteSEnergy(test_data, 23.5))
#print(DiscreteSEnergy(test_data[:, ::2], 23.5)) #calculation with only half as many points

#print(discrete_energy(test_data, 23.5))
#print(discrete_energy(test_data[:, ::2], 23.5))

#graphing the curve

def energy_graph(data, s):
    d_e_list = []
    x_list = []
    maxN = data.shape[1]
    for i in range(1, maxN // 10):
        d_e_list.append(discrete_energy(data[:, ::i], s))
        x_list.append(maxN // i)
    print(d_e_list)
    plt.subplot(2,1,1)
    plt.plot(x_list, [i for i in d_e_list])
    plt.yscale('log')
    plt.title(f"s = {s}")
    plt.show()

#discrete energy graph 1 second with the seizure data
#discrete energy graph 1 second with the non-seizure data

#patient1
for s in range(23, 26):
  energy_graph(rawsiez[:, 2000:2256], s)
  print("siezure for patient 1")

for s in range(23, 26):
  energy_graph(raw[:, 2000:2256], s)
  print("non siezure for patient 1")

#patient 2
for s in range(23, 26):
  energy_graph(rawsiez2[:, 2000:2256], s)
  print("siezure for patient 2")
for s in range(23, 26):
  energy_graph(raw2[:, 2000:2256], s)
  print("non siezure for patient 2")

#patient 3
for s in range(23, 26):
  energy_graph(rawsiez3[:, 2000:2256], s)
  print("siezure for patient 3")
for s in range(23, 26):
  energy_graph(raw3[:, 2000:2256], s)
  print("non siezure for patient 3")

Platform             macOS-15.5-arm64-arm-64bit
Python               3.11.8 | packaged by conda-forge | (main, Feb 16 2024, 20:49:36) [Clang 16.0.6 ]
Executable           /opt/anaconda3/envs/medical-data-processing/bin/python
CPU                  Apple M2 Pro (10 cores)
Memory               16.0 GiB

Core
├☑ mne               1.10.0 (latest release)
├☑ numpy             2.1.3 (unknown linalg bindings)
├☑ scipy             1.16.0
└☑ matplotlib        3.10.3 (backend=module://matplotlib_inline.backend_inline)

Numerical (optional)
├☑ sklearn           1.7.1
├☑ pandas            2.3.1
├☑ h5py              3.14.0
└☐ unavailable       numba, nibabel, nilearn, dipy, openmeeg, cupy, h5io

Visualization (optional)
└☐ unavailable       pyvista, pyvistaqt, vtk, qtpy, ipympl, pyqtgraph, mne-qt-browser, ipywidgets, trame_client, trame_server, trame_vtk, trame_vuetify

Ecosystem (optional)
└☐ unavailable       mne-bids, mne-nirs, mne-features, mne-connectivity, mne-icalabel, mne-bids-pipeline, neo,

FileNotFoundError: [Errno 2] No such file or directory: '/Users/lyz/Documents/VSCode_Workspace/StemForAll/Medical Data/chb01_04.edf'

In [ ]:

from scipy import stats
#HURST EXPONENT CALCULATIONS
#Stolen from here: https://towardsdatascience.com/introduction-to-the-hurst-exponent-with-code-in-python-4da0414ca52e

def get_hurst_exponent(time_series, max_lag=20):
    """Returns the Hurst Exponent of the time series"""

    lags = range(2, max_lag)

    # variances of the lagged differences
    tau = [np.std(np.subtract(time_series[lag:], time_series[:-lag])) for lag in lags]

    # calculate the slope of the log plot -> the Hurst Exponent
    reg = np.polyfit(np.log(lags), np.log(tau), 1)

    return reg[0]
file = "chb01_04.edf"
data = mne.io.read_raw_edf(file)
raw = data.get_data()

rawsiezh = rawsiez[:,2000:2256]
rawsiez2h = rawsiez2[:,2000:2256]
rawsiez3h = rawsiez3[:,2000:2256]

rawh=raw[:, 2000:2256]
raw2h=raw2[:, 2000:2256]
raw3h=raw3[:, 2000:2256]

#siezures for patients
s1 = get_hurst_exponent(rawsiezh[0])
s2 = get_hurst_exponent(rawsiez2h[0])
s3 = get_hurst_exponent(rawsiez3h[0])

#siezures for patients
ns1 = get_hurst_exponent(rawh) #? weird
print(ns1)
ns2 = get_hurst_exponent(raw2h[0])
ns3 = get_hurst_exponent(raw3h[0])

siezure_hurst_exp = np.array([s1,s2,s3])
print(siezure_hurst_exp)
nonsiezure_hurst_exp = np.array([ns1,ns2,ns3])
print(nonsiezure_hurst_exp)
t_stat,p_value = stats.ttest_ind(siezure_hurst_exp, nonsiezure_hurst_exp)
print(t_stat)
print(p_value)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')